# Build Datasets for RNN Price Direction Prediction

1. Get Hourly Prices for our stock universe.
2. Get daily prices for our stock universe
3. Get 15 minute prices for our stock universe.(1/5min optional)
4. Compute labels for each stock based on most signifcant linear trend scan
5. Add news headlines to respective timestamp for each dataset
6. Save all in database H5 file to model access

In [1]:
import pandas as pd
import numpy as np
import datetime
import json
from tqdm import tqdm
from functions import getBinsFrontTrend
# Ignore harmless warnings
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)

In [2]:
from pathlib import Path

In [3]:
#shutil.rmtree(path) to remove file
DATA_STORE = Path('universe.h5')
NEWS_STORE = Path('news_db.h5')

In [4]:
# Get stock universe : DOW/NASDAQ/SP500 common stocks constitutes
with open('stock_universe.json', 'r') as data_file:
    stock_universe = json.load(data_file)
    print('%d stocks are in our Universe' % len(stock_universe))

527 stocks are in our Universe


In [139]:
end_at = datetime.datetime.now() 
begin_from = end_at + datetime.timedelta(days=-5000)

for start in pd.date_range(start = begin_from, end = end_at,normalize=True,freq = '88D'):
    end = start + datetime.timedelta(days = 88)
    end = str(int((end - datetime.datetime(1970,1,1)).total_seconds()))
    start = str(int((start - datetime.datetime(1970,1,1)).total_seconds()))
    print(str(pd.to_datetime(start, unit='s'))+':'+str(pd.to_datetime(end, unit='s'))+start+"  "+end)

2007-01-18 00:00:00:2007-04-16 00:00:001169078400  1176681600
2007-04-16 00:00:00:2007-07-13 00:00:001176681600  1184284800
2007-07-13 00:00:00:2007-10-09 00:00:001184284800  1191888000
2007-10-09 00:00:00:2008-01-05 00:00:001191888000  1199491200
2008-01-05 00:00:00:2008-04-02 00:00:001199491200  1207094400
2008-04-02 00:00:00:2008-06-29 00:00:001207094400  1214697600
2008-06-29 00:00:00:2008-09-25 00:00:001214697600  1222300800
2008-09-25 00:00:00:2008-12-22 00:00:001222300800  1229904000
2008-12-22 00:00:00:2009-03-20 00:00:001229904000  1237507200
2009-03-20 00:00:00:2009-06-16 00:00:001237507200  1245110400
2009-06-16 00:00:00:2009-09-12 00:00:001245110400  1252713600
2009-09-12 00:00:00:2009-12-09 00:00:001252713600  1260316800
2009-12-09 00:00:00:2010-03-07 00:00:001260316800  1267920000
2010-03-07 00:00:00:2010-06-03 00:00:001267920000  1275523200
2010-06-03 00:00:00:2010-08-30 00:00:001275523200  1283126400
2010-08-30 00:00:00:2010-11-26 00:00:001283126400  1290729600
2010-11-

## Daily Prices 

In [174]:
import requests
import datetime
from time import sleep
from tqdm import tqdm
def get_prices(interval, file_name):
    data = pd.DataFrame()
    end_at = datetime.datetime.now() 
    begin_from = end_at + datetime.timedelta(days=-5000)
    for t in tqdm(stock_universe):
        print(t)
        print(len(data))
        for start in pd.date_range(start = begin_from, end = end_at,normalize=True,freq = '88D'):
            end = start + datetime.timedelta(days = 88)
            end = str(int((end - datetime.datetime(1970,1,1)).total_seconds()))
            start = str(int((start - datetime.datetime(1970,1,1)).total_seconds()))
            try:
                r = requests.get('https://finnhub.io/api/v1/stock/candle?symbol='+t+'&resolution='+interval+'&from='+start+'&to='+end+'&token=*****')
            except json.JSONDecodeError:
                pass
            try:
                temp_data = pd.DataFrame(r.json())
                temp_data['ticker'] = t
                data = data.append(temp_data, ignore_index=True)
                print(t+':'+str(pd.to_datetime(start, unit='s'))+':'+str(pd.to_datetime(end, unit='s')))
            except ValueError:
                pass
            except ConnectionError as error:
                print(error)
                sleep(200)
                continue
            except TimeoutError as error:
                print(error)
                sleep(200)
                continue
            sleep(0.25)
#     data.to_csv(file_name,index = False)
    data.t = pd.to_datetime(data.t, unit = 's')
    columns_name = {'t': 'time', 'o': 'Open', 'c': 'Close', 'h': 'High', 'l': 'Low', 'v': 'Volume'} 
    data= data.rename(columns = columns_name)
    data = data[['time','ticker','Open', 'Close','High', 'Low', 'Volume']]
    data = data.set_index(['time', 'ticker'])
    #data.to_csv(file_name, index = False)
    return data

In [25]:
daily_data = (pd.read_csv('daily_prices.csv',
                 engine = 'c',
                 parse_dates=['time'],
                 index_col=['time', 'ticker'],
                 infer_datetime_format=True)
     .sort_index())
print(daily_data.info(null_counts=True))
with pd.HDFStore(DATA_STORE) as store:
    store.put('prices/daily', daily_data, format = 'table', data_columns = True, encoding = 'UTF-8')

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1702575 entries, (Timestamp('2006-10-23 00:00:00'), 'A') to (Timestamp('2020-07-15 00:00:00'), 'ZTS')
Data columns (total 5 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   Open    1702575 non-null  float64
 1   Close   1702575 non-null  float64
 2   High    1702575 non-null  float64
 3   Low     1702575 non-null  float64
 4   Volume  1702575 non-null  int64  
dtypes: float64(4), int64(1)
memory usage: 71.5+ MB
None


## HOURLY PRICES

In [149]:
hourly_data = get_prices('60', 'hourly_prices.csv')

  0%|                                                                                          | 0/526 [00:00<?, ?it/s]

A
0
A:2007-01-18 00:00:00:2007-04-16 00:00:00
A:2007-04-16 00:00:00:2007-07-13 00:00:00
A:2007-07-13 00:00:00:2007-10-09 00:00:00
A:2007-10-09 00:00:00:2008-01-05 00:00:00
A:2008-01-05 00:00:00:2008-04-02 00:00:00
A:2008-04-02 00:00:00:2008-06-29 00:00:00
A:2008-06-29 00:00:00:2008-09-25 00:00:00
A:2008-09-25 00:00:00:2008-12-22 00:00:00
A:2008-12-22 00:00:00:2009-03-20 00:00:00
A:2009-03-20 00:00:00:2009-06-16 00:00:00
A:2009-06-16 00:00:00:2009-09-12 00:00:00
A:2009-09-12 00:00:00:2009-12-09 00:00:00
A:2009-12-09 00:00:00:2010-03-07 00:00:00
A:2010-03-07 00:00:00:2010-06-03 00:00:00
A:2010-06-03 00:00:00:2010-08-30 00:00:00
A:2010-08-30 00:00:00:2010-11-26 00:00:00
A:2010-11-26 00:00:00:2011-02-22 00:00:00
A:2011-02-22 00:00:00:2011-05-21 00:00:00
A:2011-05-21 00:00:00:2011-08-17 00:00:00
A:2011-08-17 00:00:00:2011-11-13 00:00:00
A:2011-11-13 00:00:00:2012-02-09 00:00:00
A:2012-02-09 00:00:00:2012-05-07 00:00:00
A:2012-05-07 00:00:00:2012-08-03 00:00:00
A:2012-08-03 00:00:00:2012-10-

  0%|▏                                                                              | 1/526 [01:13<10:39:48, 73.12s/it]

AAL
27541
AAL:2013-10-17 00:00:00:2014-01-13 00:00:00
AAL:2014-01-13 00:00:00:2014-04-11 00:00:00
AAL:2014-04-11 00:00:00:2014-07-08 00:00:00
AAL:2014-07-08 00:00:00:2014-10-04 00:00:00
AAL:2014-10-04 00:00:00:2014-12-31 00:00:00
AAL:2014-12-31 00:00:00:2015-03-29 00:00:00
AAL:2015-03-29 00:00:00:2015-06-25 00:00:00
AAL:2015-06-25 00:00:00:2015-09-21 00:00:00
AAL:2015-09-21 00:00:00:2015-12-18 00:00:00
AAL:2015-12-18 00:00:00:2016-03-15 00:00:00
AAL:2016-03-15 00:00:00:2016-06-11 00:00:00
AAL:2016-06-11 00:00:00:2016-09-07 00:00:00
AAL:2016-09-07 00:00:00:2016-12-04 00:00:00
AAL:2016-12-04 00:00:00:2017-03-02 00:00:00
AAL:2017-03-02 00:00:00:2017-05-29 00:00:00
AAL:2017-05-29 00:00:00:2017-08-25 00:00:00
AAL:2017-08-25 00:00:00:2017-11-21 00:00:00
AAL:2017-11-21 00:00:00:2018-02-17 00:00:00
AAL:2018-02-17 00:00:00:2018-05-16 00:00:00
AAL:2018-05-16 00:00:00:2018-08-12 00:00:00
AAL:2018-08-12 00:00:00:2018-11-08 00:00:00
AAL:2018-11-08 00:00:00:2019-02-04 00:00:00
AAL:2019-02-04 00:00:0

  0%|▎                                                                              | 2/526 [02:18<10:17:48, 70.74s/it]

...

100%|███████████████████████████████████████████████████████████████████████████▋| 524/526 [17:00:57<05:09, 154.68s/it]

ZM
12761097
ZM:2019-02-04 00:00:00:2019-05-03 00:00:00
ZM:2019-05-03 00:00:00:2019-07-30 00:00:00
ZM:2019-07-30 00:00:00:2019-10-26 00:00:00
ZM:2019-10-26 00:00:00:2020-01-22 00:00:00
ZM:2020-01-22 00:00:00:2020-04-19 00:00:00
ZM:2020-04-19 00:00:00:2020-07-16 00:00:00
ZM:2020-07-16 00:00:00:2020-10-12 00:00:00


100%|███████████████████████████████████████████████████████████████████████████▊| 525/526 [17:02:13<02:10, 130.95s/it]

ZTS
12764455
ZTS:2013-01-26 00:00:00:2013-04-24 00:00:00
ZTS:2013-04-24 00:00:00:2013-07-21 00:00:00
ZTS:2013-07-21 00:00:00:2013-10-17 00:00:00
ZTS:2013-10-17 00:00:00:2014-01-13 00:00:00
ZTS:2014-01-13 00:00:00:2014-04-11 00:00:00
ZTS:2014-04-11 00:00:00:2014-07-08 00:00:00
ZTS:2014-07-08 00:00:00:2014-10-04 00:00:00
ZTS:2014-10-04 00:00:00:2014-12-31 00:00:00
ZTS:2014-12-31 00:00:00:2015-03-29 00:00:00
ZTS:2015-03-29 00:00:00:2015-06-25 00:00:00
ZTS:2015-06-25 00:00:00:2015-09-21 00:00:00
ZTS:2015-09-21 00:00:00:2015-12-18 00:00:00
ZTS:2015-12-18 00:00:00:2016-03-15 00:00:00
ZTS:2016-03-15 00:00:00:2016-06-11 00:00:00
ZTS:2016-06-11 00:00:00:2016-09-07 00:00:00
ZTS:2016-09-07 00:00:00:2016-12-04 00:00:00
ZTS:2016-12-04 00:00:00:2017-03-02 00:00:00
ZTS:2017-03-02 00:00:00:2017-05-29 00:00:00
ZTS:2017-05-29 00:00:00:2017-08-25 00:00:00
ZTS:2017-08-25 00:00:00:2017-11-21 00:00:00
ZTS:2017-11-21 00:00:00:2018-02-17 00:00:00
ZTS:2018-02-17 00:00:00:2018-05-16 00:00:00
ZTS:2018-05-16 00:0

100%|████████████████████████████████████████████████████████████████████████████| 526/526 [17:04:14<00:00, 116.83s/it]


In [150]:
hourly_data = (pd.read_csv('hourly_prices.csv',
                 engine = 'c',
                 parse_dates=['time'],
                 index_col=['time', 'ticker'],
                 infer_datetime_format=True)
     .sort_index())
print(hourly_data.info(null_counts=True))
with pd.HDFStore(DATA_STORE) as store:
    store.put('prices/hourly', hourly_data, format = 'table', data_columns = True, encoding = 'UTF-8')

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 12779875 entries, (Timestamp('2007-01-18 13:00:00'), 'DISCA') to (Timestamp('2020-09-25 23:00:00'), 'ZM')
Data columns (total 5 columns):
 #   Column  Non-Null Count     Dtype  
---  ------  --------------     -----  
 0   Open    12779875 non-null  float64
 1   Close   12779875 non-null  float64
 2   High    12779875 non-null  float64
 3   Low     12779875 non-null  float64
 4   Volume  12779875 non-null  int64  
dtypes: float64(4), int64(1)
memory usage: 536.5+ MB
None


## 15 Minute Prices
- Prices returns below 15 minutes are dominated bt BID/ASK bounce.

In [175]:
data_15mins = get_prices('15', 'data_15mins.csv')

  0%|                                                                                          | 0/526 [00:00<?, ?it/s]

A
0
A:2007-01-22 00:00:00:2007-04-20 00:00:00
A:2007-04-20 00:00:00:2007-07-17 00:00:00
A:2007-07-17 00:00:00:2007-10-13 00:00:00
A:2007-10-13 00:00:00:2008-01-09 00:00:00
A:2008-01-09 00:00:00:2008-04-06 00:00:00
A:2008-04-06 00:00:00:2008-07-03 00:00:00
A:2008-07-03 00:00:00:2008-09-29 00:00:00
A:2008-09-29 00:00:00:2008-12-26 00:00:00
A:2008-12-26 00:00:00:2009-03-24 00:00:00
A:2009-03-24 00:00:00:2009-06-20 00:00:00
A:2009-06-20 00:00:00:2009-09-16 00:00:00
A:2009-09-16 00:00:00:2009-12-13 00:00:00
A:2009-12-13 00:00:00:2010-03-11 00:00:00
A:2010-03-11 00:00:00:2010-06-07 00:00:00
A:2010-06-07 00:00:00:2010-09-03 00:00:00
A:2010-09-03 00:00:00:2010-11-30 00:00:00
A:2010-11-30 00:00:00:2011-02-26 00:00:00
A:2011-02-26 00:00:00:2011-05-25 00:00:00
A:2011-05-25 00:00:00:2011-08-21 00:00:00
A:2011-08-21 00:00:00:2011-11-17 00:00:00
A:2011-11-17 00:00:00:2012-02-13 00:00:00
A:2012-02-13 00:00:00:2012-05-11 00:00:00
A:2012-05-11 00:00:00:2012-08-07 00:00:00
A:2012-08-07 00:00:00:2012-11-

  0%|▏                                                                              | 1/526 [01:26<12:37:58, 86.63s/it]

AAL
92919
AAL:2013-10-21 00:00:00:2014-01-17 00:00:00
AAL:2014-01-17 00:00:00:2014-04-15 00:00:00
AAL:2014-04-15 00:00:00:2014-07-12 00:00:00
AAL:2014-07-12 00:00:00:2014-10-08 00:00:00
AAL:2014-10-08 00:00:00:2015-01-04 00:00:00
AAL:2015-01-04 00:00:00:2015-04-02 00:00:00
AAL:2015-04-02 00:00:00:2015-06-29 00:00:00
AAL:2015-06-29 00:00:00:2015-09-25 00:00:00
AAL:2015-09-25 00:00:00:2015-12-22 00:00:00
AAL:2015-12-22 00:00:00:2016-03-19 00:00:00
AAL:2016-03-19 00:00:00:2016-06-15 00:00:00
AAL:2016-06-15 00:00:00:2016-09-11 00:00:00
AAL:2016-09-11 00:00:00:2016-12-08 00:00:00
AAL:2016-12-08 00:00:00:2017-03-06 00:00:00
AAL:2017-03-06 00:00:00:2017-06-02 00:00:00
AAL:2017-06-02 00:00:00:2017-08-29 00:00:00
AAL:2017-08-29 00:00:00:2017-11-25 00:00:00
AAL:2017-11-25 00:00:00:2018-02-21 00:00:00
AAL:2018-02-21 00:00:00:2018-05-20 00:00:00
AAL:2018-05-20 00:00:00:2018-08-16 00:00:00
AAL:2018-08-16 00:00:00:2018-11-12 00:00:00
AAL:2018-11-12 00:00:00:2019-02-08 00:00:00
AAL:2019-02-08 00:00:0

  0%|▎                                                                              | 2/526 [02:31<11:39:57, 80.15s/it]

...

100%|███████████████████████████████████████████████████████████████████████████▋| 524/526 [39:31:51<28:43, 861.88s/it]

ZM
42844335
ZM:2019-02-08 00:00:00:2019-05-07 00:00:00
ZM:2019-05-07 00:00:00:2019-08-03 00:00:00
ZM:2019-08-03 00:00:00:2019-10-30 00:00:00
ZM:2019-10-30 00:00:00:2020-01-26 00:00:00
ZM:2020-01-26 00:00:00:2020-04-23 00:00:00
ZM:2020-04-23 00:00:00:2020-07-20 00:00:00
ZM:2020-07-20 00:00:00:2020-10-16 00:00:00


100%|███████████████████████████████████████████████████████████████████████████▊| 525/526 [39:35:07<11:01, 661.93s/it]

ZTS
42856138
ZTS:2013-01-30 00:00:00:2013-04-28 00:00:00
ZTS:2013-04-28 00:00:00:2013-07-25 00:00:00
ZTS:2013-07-25 00:00:00:2013-10-21 00:00:00
ZTS:2013-10-21 00:00:00:2014-01-17 00:00:00
ZTS:2014-01-17 00:00:00:2014-04-15 00:00:00
ZTS:2014-04-15 00:00:00:2014-07-12 00:00:00
ZTS:2014-07-12 00:00:00:2014-10-08 00:00:00
ZTS:2014-10-08 00:00:00:2015-01-04 00:00:00
ZTS:2015-01-04 00:00:00:2015-04-02 00:00:00
ZTS:2015-04-02 00:00:00:2015-06-29 00:00:00
ZTS:2015-06-29 00:00:00:2015-09-25 00:00:00
ZTS:2015-09-25 00:00:00:2015-12-22 00:00:00
ZTS:2015-12-22 00:00:00:2016-03-19 00:00:00
ZTS:2016-03-19 00:00:00:2016-06-15 00:00:00
ZTS:2016-06-15 00:00:00:2016-09-11 00:00:00
ZTS:2016-09-11 00:00:00:2016-12-08 00:00:00
ZTS:2016-12-08 00:00:00:2017-03-06 00:00:00
ZTS:2017-03-06 00:00:00:2017-06-02 00:00:00
ZTS:2017-06-02 00:00:00:2017-08-29 00:00:00
ZTS:2017-08-29 00:00:00:2017-11-25 00:00:00
ZTS:2017-11-25 00:00:00:2018-02-21 00:00:00
ZTS:2018-02-21 00:00:00:2018-05-20 00:00:00
ZTS:2018-05-20 00:0

100%|████████████████████████████████████████████████████████████████████████████| 526/526 [39:45:12<00:00, 272.08s/it]


In [5]:
data_15mins = (pd.read_csv('data_15mins.csv',
                 engine = 'c',
                 parse_dates=['time'],
                 index_col=['time', 'ticker'],
                 infer_datetime_format=True)
     .sort_index())
print(data_15mins.info(null_counts=True))
with pd.HDFStore(DATA_STORE) as store:
    store.put('prices/15_min', data_15mins, format = 'table', data_columns = True, encoding = 'UTF-8')

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 42908240 entries, (Timestamp('2007-01-22 14:15:00'), 'CMCSA') to (Timestamp('2020-10-02 16:15:00'), 'ZTS')
Data columns (total 5 columns):
 #   Column  Non-Null Count     Dtype  
---  ------  --------------     -----  
 0   Open    42908240 non-null  float64
 1   Close   42908240 non-null  float64
 2   High    42908240 non-null  float64
 3   Low     42908240 non-null  float64
 4   Volume  42908240 non-null  int64  
dtypes: float64(4), int64(1)
memory usage: 1.8+ GB
None


In [15]:
len(data_15mins.index.get_level_values('ticker').unique())

524

## 5 Minute Prices

In [ ]:
data_5mins = get_prices('5', 'data_5mins.csv')

In [ ]:
data_5mins = (pd.read_csv('data_5mins.csv',
                 engine = 'c',
                 parse_dates=['time'],
                 index_col=['time', 'ticker'],
                 infer_datetime_format=True)
     .sort_index())
print(data_5mins.info(null_counts=True))
with pd.HDFStore(DATA_STORE) as store:
    store.put('prices/5_min', data_5mins, format = 'table', data_columns = True, encoding = 'UTF-8')

## 1 Minute Prices

In [ ]:
data_1mins = get_prices('1', 'data_1mins.csv')

In [ ]:
data_1mins = (pd.read_csv('data_1mins.csv',
                 engine = 'c',
                 parse_dates=['time'],
                 index_col=['time', 'ticker'],
                 infer_datetime_format=True)
     .sort_index())
print(data_1mins.info(null_counts=True))
with pd.HDFStore(DATA_STORE) as store:
    store.put('prices/1_min', data_1mins, format = 'table', data_columns = True, encoding = 'UTF-8')

In [ ]:
#shutil.rmtree(path) to remove file

Save all downloads on EC2 S3!

In [ ]:
#df = df[df.index.get_level_values('symbol').isin(list)]
#equivalent to df[df.symbol == 'AAPL'] or df.xs('AAPL', level='symbol') or df.loc[(slice(None), 'AAPL'), :]
# new_df = df.xs('nasdaq', level='symbol').copy()
#data_15mins[data_15mins.index.get_level_values('ticker').isin(stock_universe)]

## Update Prices

In [5]:
import requests
from time import sleep
from tqdm import tqdm
def update_prices(interval, table):
    data = pd.DataFrame()
    with pd.HDFStore(DATA_STORE, mode = 'r+') as store:
        print(store.keys()) 
        #table = str(input('Enter table key printed above: '))
        df = store[table].sort_index()
        end_at = datetime.datetime.now() 
        begin_from = df.index.get_level_values(0).max() + datetime.timedelta(1)
        for t in tqdm(stock_universe):
            print(t)
            print(len(data))
            for start in pd.date_range(start = begin_from, end = end_at,normalize=True,freq = '20D'):
                end = start + datetime.timedelta(days = 19)
                end = str(int((end - datetime.datetime(1970,1,1)).total_seconds()))
                start = str(int((start - datetime.datetime(1970,1,1)).total_seconds()))
                try:
                    r = requests.get('https://finnhub.io/api/v1/stock/candle?symbol='+t+'&resolution='+interval+'&from='+start+'&to='+end+'&token=*****')
                except json.JSONDecodeError:
                    pass
                try:
                    temp_data = pd.DataFrame(r.json())
                    temp_data['ticker'] = t
                    data = data.append(temp_data, ignore_index=True)
                    print(t+':'+str(pd.to_datetime(start, unit='s'))+':'+str(pd.to_datetime(end, unit='s')))
                except ValueError:
                    pass
                except ConnectionError as error:
                    print(error)
                    sleep(200)
                    continue
                except TimeoutError as error:
                    print(error)
                    sleep(200)
                    continue
                sleep(0.25)
        data.t = pd.to_datetime(data.t, unit = 's')
        columns_name = {'t': 'time', 'o': 'Open', 'c': 'Close', 'h': 'High', 'l': 'Low', 'v': 'Volume'} 
        data= data.rename(columns = columns_name)
        data = data[['time','ticker','Open', 'Close','High', 'Low', 'Volume']]
        data = data.set_index(['time', 'ticker'])
        data.to_csv('data.csv', index = False)
        store.put(table, data, encoding = 'UTF-8',format = 'table',append = True)
    return 1

In [6]:
update_prices('D','/prices/daily')

['/prices/15_min', '/prices/daily', '/prices/hourly']


  0%|                                                                                          | 0/527 [00:00<?, ?it/s]

A
0
A:2020-10-15 00:00:00:2020-11-03 00:00:00


  0%|▏                                                                                 | 1/527 [00:01<09:46,  1.11s/it]

AAL
7
AAL:2020-10-15 00:00:00:2020-11-03 00:00:00


...

ZM
3650
ZM:2020-10-15 00:00:00:2020-11-03 00:00:00


100%|███████████████████████████████████████████████████████████████████████████████▊| 526/527 [08:46<00:01,  1.00s/it]

ZTS
3657
ZTS:2020-10-15 00:00:00:2020-11-03 00:00:00


100%|████████████████████████████████████████████████████████████████████████████████| 527/527 [08:47<00:00,  1.00s/it]


1

In [7]:
update_prices('60', '/prices/hourly')

['/prices/15_min', '/prices/daily', '/prices/hourly']


  0%|                                                                                          | 0/527 [00:00<?, ?it/s]

A
0
A:2020-10-15 00:00:00:2020-11-03 00:00:00


  0%|▏                                                                                 | 1/527 [00:03<33:58,  3.88s/it]

AAL
58
AAL:2020-10-15 00:00:00:2020-11-03 00:00:00


...

ZM
24986


100%|███████████████████████████████████████████████████████████████████████████████▊| 526/527 [05:50<00:00,  1.65it/s]

ZTS
24986


100%|████████████████████████████████████████████████████████████████████████████████| 527/527 [05:50<00:00,  1.50it/s]


1

In [8]:
update_prices('15', '/prices/15_min')

['/prices/15_min', '/prices/daily', '/prices/hourly']


  0%|                                                                                          | 0/527 [00:00<?, ?it/s]

A
0
A:2020-10-15 00:00:00:2020-11-03 00:00:00


  0%|▏                                                                                 | 1/527 [00:01<09:18,  1.06s/it]

AAL
193
AAL:2020-10-15 00:00:00:2020-11-03 00:00:00


...

ZM
85382
ZM:2020-10-15 00:00:00:2020-11-03 00:00:00


100%|███████████████████████████████████████████████████████████████████████████████▊| 526/527 [06:11<00:01,  1.08s/it]

ZTS
85709
ZTS:2020-10-15 00:00:00:2020-11-03 00:00:00


100%|████████████████████████████████████████████████████████████████████████████████| 527/527 [06:11<00:00,  1.42it/s]


1

# Access From EC2 S3

In [2]:
def access_s3(interval):
    import h5py
    import s3fs
    data = pd.DataFrame()
    s3 = s3fs.S3FileSystem(anon=False, key='*****', 
                           secret='*******',
                           client_kwargs={'region_name':'us-west-2'})
    with h5py.File(s3.open("charlanguagemodeldata/universe.h5", 'rb'), 'r', lib_version='latest') as f:
        print(list(f.keys()))
        print(list(f.get('prices').keys())) 
        get_columns =['time', 'ticker', 'Open', 'Close', 'High', 'Low', 'Volume']
        data = pd.DataFrame(f['prices/'+interval+'/table'].value,columns=get_columns)
        data[data.columns[data.dtypes == object]] = data.select_dtypes([object]).stack().str.decode('utf-8').unstack()
        data.time = pd.to_datetime(data.time, unit='ns')
        data = data.set_index(['time', 'ticker'])
    f.close()
    return data

In [3]:
daily_prices = access_s3('daily')

['prices']
['15_min', 'daily', 'hourly']


In [12]:
daily_prices.iloc[daily_prices.index.get_loc('AAPL'), :]['Close']

ticker  time      
AAPL    2006-10-23     2.910000
        2006-10-24     2.890000
        2006-10-25     2.920000
        2006-10-26     2.940000
        2006-10-27     2.870000
                        ...    
        2020-07-09    95.682503
        2020-07-10    95.919998
        2020-07-13    95.477501
        2020-07-14    97.057503
        2020-07-15    97.724998
Name: Close, Length: 3456, dtype: float64

In [6]:
daily_prices['tL'] = np.nan
daily_prices['trend'] = np.nan
for i in tqdm(daily_prices.index.get_level_values(level = 'ticker').unique()):
    df = daily_prices.xs(i, level='ticker')
    df = getBinsFrontTrend(df.index, df.Close, [3,21,3])
    daily_prices.iloc[daily_prices.index.get_loc(i), :]['trend'] = df['bin']
    daily_prices.iloc[daily_prices.index.get_loc(i), :]['tL'] = df['t1']

100%|█████████████████████████████████████████████████████████████████████████████| 523/523 [11:40:02<00:00, 80.31s/it]


## Prepare Model Data (Headlines + Prices)

In [10]:
idx = pd.IndexSlice

#### Get News Collected

In [5]:
with pd.HDFStore(NEWS_STORE) as store:
    keys = list(store.keys())
    news = pd.DataFrame()
    for i in tqdm(keys): 
        news = news.append(store[i])  

news = news.reset_index().set_index('versionCreated')
news.index = news.index.strftime('%Y-%m-%d %H:%M:%S')
news.index = pd.to_datetime(news.index)
news = news.drop(['index','storyId', 'sourceCode'], axis = 1)
news.index.name = 'time'
news.info()

100%|████████████████████████████████████████████████████████████████████████████████| 532/532 [04:22<00:00,  2.03it/s]


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 962629 entries, 2020-09-01 21:20:54 to 2020-10-16 16:06:34
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    962629 non-null  object
 1   ticker  962629 non-null  object
dtypes: object(2)
memory usage: 22.0+ MB


In [6]:
# Resample News be Frequency
daily_news = news.copy().groupby('ticker').resample('D').agg({'text': ' '.join})
hourly_news = news.copy().groupby('ticker').resample('H').agg({'text': ' '.join})
min_15_news = news.copy().groupby('ticker').resample('15min').agg({'text': ' '.join})

In [8]:
start = min_15_news.index.get_level_values(1).min().strftime('%Y-%m-%d %H:%M:%S')
end = min_15_news.index.get_level_values(1).max().strftime('%Y-%m-%d %H:%M:%S')
print(start, end)
start = '2019-07-08 13:00:00'
end = '2020-10-14 20:00:00'

2019-07-08 07:15:00 2020-10-15 16:00:00


#### Daily Model Data Processing

In [100]:
daily['returns'] = daily.Close.groupby('ticker').pct_change().shift(-1)
daily['label'] = daily.returns.where(daily.returns > 0, -1).where(daily.returns < 0, 1)

In [101]:
intersect = daily.index.intersection(daily_news.index)

In [102]:
daily_news = daily_news.loc[intersect, :].sort_index()
daily = daily.loc[intersect, :].sort_index()

In [103]:
daily = daily_news.join(daily[['Open', 'Close', 'returns', 'label']])

In [104]:
daily.text = daily.text.replace('', np.nan)

In [143]:
daily.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 154146 entries, ('A', Timestamp('2019-07-11 00:00:00')) to ('ZTS', Timestamp('2020-10-02 00:00:00'))
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   text     120105 non-null  object 
 1   Open     154146 non-null  float64
 2   Close    154146 non-null  float64
 3   returns  154103 non-null  float64
 4   label    154146 non-null  float64
dtypes: float64(4), object(1)
memory usage: 11.5+ MB


In [144]:
with pd.HDFStore(DATA_STORE) as store:
    store.put('model_data/daily', daily)

#### Hourly Model Data Processing

In [148]:
# # last price after 1 hr compared to first price when news was released.
def o_c_pct_change(df):
    return (df.Close.shift(-1) - df.Open)/ df.Open

hourly['returns'] = hourly.groupby('ticker', group_keys=False).apply(o_c_pct_change)
hourly['label'] = hourly.returns.where(hourly.returns > 0, -1).where(hourly.returns < 0, 1)

In [149]:
hourly_intersect = hourly.index.intersection(hourly_news.index)

In [151]:
hourly_news = hourly_news.loc[hourly_intersect, :].sort_index()
hourly = hourly.loc[hourly_intersect, :].sort_index()

In [153]:
hourly = hourly_news.join(hourly[['Open', 'Close', 'returns', 'label']])

In [159]:
hourly.text = hourly.text.replace('', np.nan)

In [160]:
hourly.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1293713 entries, ('A', Timestamp('2019-07-11 13:00:00')) to ('ZTS', Timestamp('2020-10-02 20:00:00'))
Data columns (total 5 columns):
 #   Column   Non-Null Count    Dtype  
---  ------   --------------    -----  
 0   text     235037 non-null   object 
 1   Open     1293713 non-null  float64
 2   Close    1293713 non-null  float64
 3   returns  1293589 non-null  float64
 4   label    1293713 non-null  float64
dtypes: float64(4), object(1)
memory usage: 94.3+ MB


In [162]:
with pd.HDFStore(DATA_STORE) as store:
    store.put('model_data/hourly', hourly)

#### 15 Minute Model Data Processing

In [20]:
min_15_news.text = min_15_news.text.replace('', np.nan)
min_15_news = min_15_news.dropna()

In [22]:
def o_c_pct_change(df):
    return (df.Close.shift(-1) - df.Open)/ df.Open

min_15['returns'] = min_15.groupby('ticker', group_keys=False).apply(o_c_pct_change)
min_15['label'] = min_15.returns.where(min_15.returns > 0, -1).where(min_15.returns < 0, 1)

In [23]:
min15_intersect = min_15.index.intersection(min_15_news.index)

In [25]:
min_15_news = min_15_news.loc[min15_intersect, :].sort_index()
min_15 = min_15.loc[min15_intersect, :].sort_index()

In [26]:
min_15 = min_15_news.join(min_15[['Open', 'Close','returns', 'label']])

In [27]:
min_15.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 263771 entries, ('A', Timestamp('2019-07-11 16:30:00')) to ('ZTS', Timestamp('2020-10-14 16:45:00'))
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   text     263771 non-null  object 
 1   Open     263771 non-null  float64
 2   Close    263771 non-null  float64
 3   returns  263747 non-null  float64
 4   label    263771 non-null  float64
dtypes: float64(4), object(1)
memory usage: 21.2+ MB


In [28]:
with pd.HDFStore(DATA_STORE) as store:
    store.put('model_data/15_min', min_15)

In [44]:
oc = ['Open', 'Close']
with pd.HDFStore(DATA_STORE) as store:
#     daily = store['prices/daily'].swaplevel().sort_index()
#     daily = daily[~daily.index.duplicated()]
#     hourly = store['prices/hourly'].swaplevel().sort_index()
#     hourly = hourly[~hourly.index.duplicated()]
    min_15 = (store['prices/15_min'].sort_index(level = 0,sort_remaining=0)
                                    .loc[idx[start:end, :], oc]
                                    .swaplevel().sort_index())
    min_15 = min_15[~min_15.index.duplicated()]

In [46]:
min_15.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 4648952 entries, ('A', Timestamp('2019-07-08 13:30:00')) to ('ZTS', Timestamp('2020-10-14 20:00:00'))
Data columns (total 2 columns):
 #   Column  Dtype  
---  ------  -----  
 0   Open    float64
 1   Close   float64
dtypes: float64(2)
memory usage: 98.3+ MB


### Fix HDFS Datasets Format

Model Data where saved without specifying format of set to be table that allows flexible sorting and indexing while streaming data from the file.h5 from remote or cloud server.

## Feature Sample Class for Trend Scanning

### Daily

In [4]:
idx = pd.IndexSlice

In [8]:
start = news.index.min().replace(hour=0,minute=0,second=0,microsecond=0).strftime('%Y-%m-%d')
end = news.index.max().replace(hour=0,minute=0,second=0,microsecond=0).strftime('%Y-%m-%d')
unid_ticks = news.ticker.unique()
print(start)

2019-07-08


In [28]:
with pd.HDFStore(DATA_STORE) as store:
    daily = store['prices/daily'].sort_index().loc[idx['2019':, :], 'Close']
    daily = daily[~daily.index.duplicated()].unstack('ticker')